# 대응점 문제(Correspondence problem)
대응점 문제는 이웃한 영상에 나타난 같은 물체의 같은 곳을 쌍으로 묶어주는 것입니다.

대응점 문제를 해결하면 파노라마, 물체 추적, 카메라 캘리브레이션, 스테레오 비전 등에 응용 가능합니다.

### 대응점 문제 해결 아이디어 -> 지역 특징을 통한 특징점 추출
지역 특징의 조건은 아래와 같습니다.
1. 반복성(repeatability): 같은 물체가 서로 다른 두 영상에 나타났을 때 첫 영상에서 검출된 특징점이 다음 영상에서도 같은 위치에서 높은 확률로 검출되어야 합니다.
2. 불변성(invariance): 물체에 이동, 회전, 스케일, 조명 변환이 일어나도 특징점의 값은 비슷해야 합니다.
3. 분별력(discriminative power): 물체의 다른 곳의 특징과 두드러지게 달라야 합니다. 그렇지 않으면 다른 곳과 매칭될 수 있기 때문입니다.
4. 지역성(locality): 작은 영역으로 특징 벡터를 추출해야 물체의 가림(occlusion)이 발생해도 안정적인 매칭이 가능하다.
5. 적당한 양: 대응점은 오류를 내포할 수 있으므로 특징점이 더 많으면 더 정확한 추적이 가능하다. 반면 특징점이 너무 많으면 계산 시간이 과다해진다.
6. 계산 효율: 초당 몇 프레임 이상을 처리해야 하는 실시간 조건을 필요로 하는 상황에서는 계산 시간이 매우 중요하다.

대응점 문제에 관해서는 지역 특징이 독보적인 성능을 보장합니다.

반면 사진에서 특정 물체의 영역을 배경으로 대치하는 편집 과정에서는 지역 특징이 쓸모 없습니다. 여기에서는 GrabCut 같은 영역 분할 알고리즘이 효과적입니다.

성공적인 컴퓨터 비전 시스템을 구축하려면 문제에 대한 정확한 이해를 바탕으로 적절한 알고리즘을 선택하는 안목이 중요합니다.

# 이동과 회전에 불변한 지역 특징

모라벡 알고리즘은 컴퓨터 비전 초기 특징점 감지 알고리즘 중 하나이다.

모라벡 알고리즘은 이미지 내에서 조건들을 만족하는 훌륭한 지역 특징을 찾는 방법을 제시하였으나, 실제 세계의 영상에서 적용은 한계가 있었다.

해리스는 2차 모멘트 행렬을 통한 특징점 검출을 통해 알고리즘을 크게 개선하였다.

### 해리스 특징점
해리스가 제시한 2차 모멘트 행렬 $A$는 아래와 같다. 수식의 유도과정은 도서를 참고한다.

> $A = \begin{pmatrix} G\circledast d_{y}^{2} & G\circledast d_{y}d_{x} \\ G\circledast d_{y}d_{x} & G\circledast d_{x}^{2} \end{pmatrix}$
* $G$: 2차 가우시안 필터 $G(x,y)$
* $d_{y}(y,x)$: $y$ 방향의 미분값($y$ 방향 그레이디언트)
* $d_{x}(y,x)$: $x$ 방향의 미분값($x$ 방향 그레이디언트)
* $\circledast$: 컨볼루션 연산자
* 예를들어 $G\circledast d_{y}^{2}$는 $d_{y}$($y$ 방향 그레이디언트)의 제곱에 가우시안 필터 $G$를 컨볼루션 하라는 의미


행렬 $A$는 여러 가지 유용한 특성이 있다.
1. 정수뿐 아니라 실수 계산도 가능하다.
2. $A$는 어떤 화소 주위의 영상 구조를 표현하고 있어, $A$만 분석하면 지역 특징 여부를 판단할 수 있다.

해리스는 $A$의 고윳값을 보고 지역 특징으로 좋은 정도를 측정하는 기법을 제안한다. (2차원 행렬의 고윳값 계산과정은 선형대수 참고)

해리스는 두 고윳값이 모두 큰 경우에 지역 특징으로 훌륭하다는 사실에 기반하여 특징 가능성 값 $C$의 수식을 아래와 같의 정의하였다.

> $C = \lambda_{1}\lambda_{2} - k(\lambda_{1}+\lambda_{2})^{2}$

$k$값은 특징점 감지의 민감도를 조정하는데, 여기서 $k$값은 두 고윳값의 곱과 합의 제곱 사이의 밸런스를 맞추는 데 중요한 역할을 합니다.

즉, $k$값은 $\lambda_{1}$과 $\lambda_{2}$가 모두 클 때, $C$ 값이 최대가 되도록 조절합니다.

실험적 과정을 통해서 $k = 0.04$로 설정할 때 적절하다고 알려져 있습니다.

그러나 위의 식은 고윳값을 계산하는 시간이 필요합니다. 모든 화소에 대해서는 이 과정이 좋지 못한 계산 효율을 야기합니다.

$A = \begin{pmatrix} p & r \\ r & q \end{pmatrix}$이라 하면 고윳값의 합은 $\lambda_{1}+\lambda_{2} = p+q$이고 곱은 $\lambda_{1}\lambda_{2} = pq-r^{2}$입니다. (증명 과정은 선형대수 참고)

이에 기반하여 $C$에 대한 식을 아래와 같이 쓸수 있습니다.

> $C = (pq-r^{2})-k(p+q)^{2}$

위의 식은 고윳값 계산 과정을 생략하여 빠른 계산이 가능합니다.